<a href="https://colab.research.google.com/github/lucaskydelima/Optimization-with-Python-Pyomo/blob/main/S7Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pyomo

In [6]:
import os
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [ ]:
model = pyo.ConcreteModel()

#Sets
model.i = pyo.RangeSet(1,3)

#Parameters
D = pd.read_excel('S7P3_Data.xlsx',sheet_name='Sheet1',header=0,index_col=0,usecols='A:J',nrows=3)
#D['Co'][1]

model.Vt = pyo.Param(initialize = 350)
Vt = model.Vt
model.DPt = pyo.Param(initialize = 400)
DPt = model.DPt
model.Wmax = pyo.Param(initialize = 2950)
Wmax = model.Wmax
model.Npmax = pyo.Param(initialize = 3)
Npmax = model.Npmax
model.Nsmax = pyo.Param(initialize = 3)
Nsmax = model.Nsmax

#Decision Variables
#Positive Reals
model.x = pyo.Var(model.i,domain=pyo.NonNegativeReals,bounds = (0,1))
x = model.x
model.v = pyo.Var(model.i,domain=pyo.NonNegativeReals,bounds = (0,Vt))
v = model.v
model.w = pyo.Var(model.i,domain=pyo.NonNegativeReals,bounds = (0,Wmax))
w = model.w
model.DP = pyo.Var(model.i,domain=pyo.NonNegativeReals,bounds = (0,DPt))
DP = model.DP

def Power_bound(model,i):
  return (0,D['Pmax'][i])

model.P = pyo.Var(model.i,domain=pyo.NonNegativeReals,bounds = Power_bound)
P = model.P

#Integer Variables
model.Np = pyo.Var(model.i,domain = pyo.NonNegativeIntegers, bounds=(0,3))
Np = model.Np
model.Ns = pyo.Var(model.i,domain = pyo.NonNegativeIntegers, bounds=(0,3))
Ns = model.Ns

#Binary Variable
model.z = pyo.Var(model.i,domain=pyo.Binary)
z = model.z

#Objective Function
def Objective_rule(model):
  return sum((D['Co'][i]+(D['Cp'][i]*P[i]))*Np[i]*Ns[i]*z[i] for i in model.i)
model.Objf = pyo.Objective(rule=Objective_rule,sense=pyo.minimize)

#Constraints
def Constraint1(model,i):
  return sum(x[i] for i in model.i) == 1
model.Const1 = pyo.Constraint(model.i,rule=Constraint1)

def Constraint2(model,i):
  return P[i] - D['alpha'][i]*(w[i]/Wmax)**3 - D['beta'][i]*(w[i]/Wmax)**2*v[i] - D['gamma'][i]*(w[i]/Wmax)*(v[i]**2) == 0
model.Const2 = pyo.Constraint(model.i,rule=Constraint2)

def Constraint3(model,i):
  return DP[i] - D['a'][i]*(w[i]/Wmax)**2 - D['b'][i]*(w[i]/Wmax)*v[i] - D['c'][i]*(v[i]**2) == 0
model.Const3 = pyo.Constraint(model.i,rule=Constraint3)

def Constraint4(model,i):
  return v[i]*Np[i] - x[i]*Vt == 0
model.Const4 = pyo.Constraint(model.i,rule=Constraint4)

def Constraint5(model,i):
  return DPt*z[i] - DP[i]*Ns[i] == 0
model.Const5 = pyo.Constraint(model.i,rule=Constraint5)

def Constraint6(model,i):
  return P[i] <= z[i]*D['Pmax'][i]
model.Const6 = pyo.Constraint(model.i, rule=Constraint6)

def Constraint7(model,i):
  return DP[i] <= z[i]*DPt
model.Const7 = pyo.Constraint(model.i, rule=Constraint7)

def Constraint8(model,i):
  return v[i] <= z[i]*Vt
model.Const8 = pyo.Constraint(model.i, rule=Constraint8)

def Constraint9(model,i):
  return w[i] <= z[i]*Wmax
model.Const9 = pyo.Constraint(model.i, rule=Constraint9)

def Constraint10(model,i):
  return Np[i] <= z[i]*Npmax
model.Const10 = pyo.Constraint(model.i, rule=Constraint10)

def Constraint11(model,i):
  return Ns[i] <= z[i]*Nsmax
model.Const11 = pyo.Constraint(model.i, rule=Constraint11)

def Constraint12(model,i):
  return x[i]<=z[i]
model.Const12 = pyo.Constraint(model.i, rule=Constraint12)

# Solve
os.environ['NEOS_EMAIL'] = 'lucaskydelima@gmail.com'
solver_manager = pyo.SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='bonmin')

print(results)
print('Objective Function= ',model.Objf())
for i in model.i:
  print('Number of Parallel Lines at Level ',i,'is = ',Np[i]())
  print('Number of Pumps in Each Line at Level ',i,'is = ',Ns[i]())
